<a href="https://colab.research.google.com/github/jtao22/PythonAI/blob/main/StockMarket/stocksentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 5.9MB/s 


In [ ]:
#import
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as SIA
import pandas as pd
import numpy as np
import re
from textblob import TextBlob as TB
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
#upload data
from google.colab import files
files.upload()

In [ ]:
newsdata = pd.read_csv('Combined_News_DJIA.csv')
tabledata = pd.read_csv('upload_DJIA_table.csv')
#merge on date
merged = newsdata.merge(tabledata, how = 'inner', on = 'Date', left_index = True)
merged #check

In [ ]:
#flatten
headlines = []
for i in range(0, len(merged.index)):
  headlines.append(' '.join(str(string) for string in merged.iloc[i, 2:27]))
#filter/clean data
cleaned_data = []
for i in range(0, len(headlines)):
  cleaned_data.append(re.sub("b[(')]",'',headlines[i]))
  cleaned_data[i] = re.sub('b[(")]', '', cleaned_data[i])
  cleaned_data[i] = re.sub("\'", '', cleaned_data[i])
merged['Cleaned News'] = cleaned_data

In [ ]:
#subjectivity function (how biased the text is)
def getSub(text):
  return TB(text).sentiment.subjectivity
#polarity function (how opinionated the text is)
def getPol(text):
  return TB(text).sentiment.polarity
#sentiment score function
def getSIA(text):
  sia = SIA()
  sentiment = sia.polarity_scores(text)
  return sentiment

In [ ]:
#add new features
merged['Subjectivity'] = merged['Cleaned News'].apply(getSub)
merged['Polarity'] = merged['Cleaned News'].apply(getPol)

In [ ]:
#get sentiment scores/day
compound = []
pos = []
neg = []
neu = []
S_I_A = 0
for i in range(0,len(merged['Cleaned News'])):
  S_I_A = getSIA(merged['Cleaned News'][i])
  compound.append(S_I_A['compound'])
  pos.append(S_I_A['pos'])
  neg.append(S_I_A['neg'])
  neu.append(S_I_A['neu'])
#store sentiment scores
merged['Compound'] = compound
merged['Positive'] = pos
merged['Negative'] = neg
merged['Neutral'] = neu

In [ ]:
#create data to train architecture
X = np.array(merged[['Open', 'High', 'Low', 'Volume', 'Subjectivity', 'Polarity', 'Compound', 'Positive', 'Negative', 'Neutral']])
Y = np.array(merged['Label'])
trainX, testX, trainY, testY = tts(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
#create architecture
arch = LDA().fit(trainX, trainY)
#predict
predictions = arch.predict(testX)
#check
print(classification_report(testY, predictions))

              precision    recall  f1-score   support

           0       0.87      0.79      0.83       193
           1       0.82      0.89      0.85       205

    accuracy                           0.84       398
   macro avg       0.84      0.84      0.84       398
weighted avg       0.84      0.84      0.84       398

